## Исследование надёжности заёмщиков

### Шаг 1. Изучение общей информации.

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
print(data)
print(data.info())

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

### Вывод

Таблица открыта и изучена. Таблица имеет 21525 строк и 12 столбцов. Визуальное изучение таблицы показало наличие отрицательных значений, пропущенных значений, слов написанных в разном регистре.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
print(data['gender'].value_counts())
print(data['dob_years'].value_counts())
print(data['children'].value_counts())
print("============================================")
data['gender'] = data['gender'].replace('XNA', np.NaN)
data['dob_years'] = data['dob_years'].replace(0, np.NaN)
data['children'] = data['children'].replace(20, np.NaN)
print("============================================")
print(data.isnull().sum())
data.dropna(subset = ['dob_years','gender', 'children'], inplace = True)
print(data.isnull().sum())
# print(data.info())
print("============================================")

data['days_employed'] = data['days_employed'].apply(abs) # Переводим отрицательные значения в положительные
# print(data.head(10)) #Проверяем выполнился ли перевод отрицательных значений в положительные в первых 10 строках

print('==============================')
def checkDaysEmployed(row): #Создаем функцию проверки максимально возможного числа рабочих дней
    # В РФ можно работать с 14 лет
    # 248 рабочих дней в году, но мы берем 365, т.к. похоже в датасете используется 365
    maxnumber_of_workingdays = (row['dob_years'] - 14) * 365
    if(row['days_employed'] > maxnumber_of_workingdays):  #Если указанный трудовой стаж больше максимально возможного числа рабочих дней
        row['days_employed'] = row['days_employed']/24 #Значит, возможно, стаж указан в часах. Поделим на 24, чтобы получить число дней.
        return row
    return row

data = data.apply(checkDaysEmployed, axis=1) #Применяем функцию к каждой строке 
print(data.head(10)) #Проверяем выполнился ли перевод подозрительно высоких значений стажа в дни в первых 10 строках

print("============================================")
print(data.isnull().sum()) # Находим столбцы, в которых имеются строки с пропущенными данными
total_income_median = data['total_income'].median() # Находим медианные значения в столбце'total_income'
days_employed_median = data['days_employed'].median() # Находим медианные значения в столбце 'days_employed'
print(total_income_median)
print("============================================")
data['total_income'] = data['total_income'].fillna(total_income_median) # Заменяем пустые строки в столбце'total_income' на медианное значение 
data['days_employed'] = data['days_employed'].fillna(days_employed_median) # Заменяем пустые строки в столбце'days_employed'на медианное значение 
print(data.isnull().sum()) # Проверяем остались ли столбцы, в которых имеются строки с пропущенными данными
print("============================================")

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
children              76
days_employed       2174
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender          

### Вывод

* В столбцах 'days_employed' и 'total_income' обнаружены пропущенные данные. В них записан NaN. В столбце 'gender' обнаружено непригодное для анализа значение 'XNA' в единственном экземпляре. В столбце 'dob_years' обнаружено значение возраста 0 лет, встречающееся 101 раз. В столбце 'children' обнаружено значение 20 детей, встречающееся 76 раз и, судя по всему, является опечаткой.
* Пропуски в столбцах 'days_employed' и 'total_income' заполнили медианными значениями; пропуски в столбцах 'gender', 'dob_years' и 'children' удалены, в виду их небольшого количества и невозможности их заполнения. 
* Пропущенные значения могут появляться из-за невнимательности (человеческий фактор), либо по техническим причинам. Также при сборе данных некоторые респонденты не желают указывать некоторые данные.

### Замена типа данных

In [5]:
print(data.info()) # Получаем сводную информацию о таблице
data['total_income'] = data['total_income'].astype('int') # Переводим данные столбца 'total_income' из float в int для удобства
data['days_employed'] = data['days_employed'].astype('int')# Переводим данные столбца 'days_employed' из float в int 
data['children'] = data['children'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
print(data.info()) # Получаем сводную информацию о таблице, чтобы проверить изменился ли тип данных
print(data.head(10)) #Проверяем выполнился ли перевод дробных чисел в целые в первых 10 строках

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21348 entries, 0 to 21524
Data columns (total 12 columns):
children            21348 non-null float64
days_employed       21348 non-null float64
dob_years           21348 non-null float64
education           21348 non-null object
education_id        21348 non-null int64
family_status       21348 non-null object
family_status_id    21348 non-null int64
gender              21348 non-null object
income_type         21348 non-null object
debt                21348 non-null int64
total_income        21348 non-null float64
purpose             21348 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21348 entries, 0 to 21524
Data columns (total 12 columns):
children            21348 non-null int64
days_employed       21348 non-null int64
dob_years           21348 non-null int64
education           21348 non-null object
education_id        21348 non-null int64
fami

### Вывод

Для того, чтобы перевести данные в нужный тип, применили метод astype(). В качестве аргумента передали строку int.

### Обработка дубликатов

In [6]:
print(data.duplicated()) # Запрашиваем отображение всех строк таблицы. Там, где есть дубликаты, будет значение True, где дубликата нет - False .
print ('Дубликатов в таблице:', data.duplicated().sum()) # Получаем количество строк-дубликатов
data['education'] = data['education'].str.lower() # Все символы в строках столбца 'education' приводим к нижнему регистру  
print(data['education'].head(20)) # Проверяем приведение к единому нижнему регистру
print ('Дубликатов в таблице:', data.duplicated().sum()) # Вновь запрашиваем количество строк-дубликатов; из-за приведения к общему регистру число повторов увеличилось.
data = data.drop_duplicates().reset_index(drop= True) # Удаляем дубликаты, используя метод reset_index(), при этом вместе с повторяющимися строками удаляются их индексы. 
print ('Дубликатов в таблице:', data.duplicated().sum()) # Проверяем остались ли дубликаты

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21348, dtype: bool
Дубликатов в таблице: 54
0                  высшее
1                 среднее
2                 среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 среднее
8                  высшее
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
15                среднее
16                среднее
17                 высшее
18                среднее
19                среднее
Name: education, dtype: object
Дубликатов в таблице: 71
Дубликатов в таблице: 0


### Вывод

* Приведение данных к общему регистру в столбце 'education' привело к увеличению количества строк-дубликатов. Для нахождения и удаления дубликатов использовали методы duplicated() и drop_duplicates(), так как они применяются ко всему датафрейму. Метод value_counts(), который анализирует столбец, в данном случае, не применяли. 

* Основные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

### Лемматизация

In [7]:
from pymystem3 import Mystem # Импортируем библиотеку pymystem3 
m = Mystem()

def myLemma(text): # Создаем функцию, которая
    lemmText = ''.join(m.lemmatize(text)) # лемматизирует текст, итоговый результат склеивает вызовом метода join()
    return lemmText.rstrip('\n') # возвращает результат, удаляя '\n' в конце каждой строки  

data['purpose'] = data['purpose'].apply(myLemma) # В ячейки заданного столбца записываем значения, возвращаемые функцией при помощи метода apply() 
print(data['purpose'].head(20))

from collections import Counter
def cout_unique_values(column):
    lemmas = []
    for text in column:
        lemmas += m.lemmatize(text)
    return Counter(lemmas)


print(cout_unique_values(data['purpose']))

0                              покупка жилье
1                    приобретение автомобиль
2                              покупка жилье
3                 дополнительный образование
4                            сыграть свадьба
5                              покупка жилье
6                           операция с жилье
7                                образование
8                      на проведение свадьба
9                    покупка жилье для семья
10                      покупка недвижимость
11         покупка коммерческий недвижимость
12                           сыграть свадьба
13                   приобретение автомобиль
14                покупка жилой недвижимость
15    строительство собственный недвижимость
16                              недвижимость
17                строительство недвижимость
18           на покупка подержать автомобиль
19                на покупка свой автомобиль
Name: purpose, dtype: object
Counter({' ': 33312, '\n': 21277, 'недвижимость': 6306, 'покупка': 5851

### Вывод

Импортировали библиотеку pymystem3. Создали функцию, которая лемматизирует текст, итоговый результат склеивает вызовом метода join() и возвращает результат, удаляя '\n' в конце каждой строки. В ячейки заданного столбца записали значения, возвращаемые функцией при помощи метода apply().

### Категоризация данных

In [8]:
 def categorize(purpose): # Проводим категоризацию. На вход функции попадает цель кредита, а возвращает категорию 
    if 'образование' in purpose:
         return 'образование'
    if 'недвижимость' in purpose:
         return 'недвижимость'
    if 'автомобиль' in purpose:
         return 'автомобиль'
    if 'свадьба' in purpose:
         return 'свадьба'
    if 'жилье' in purpose:
         return 'жилье'
    else: 
         return purpose

data['purpose'] = data['purpose'].apply(categorize) # В ячейки заданного столбца записываем категории, возвращаемые функцией 
data['purpose'] = data['purpose'].astype('category') # Проверка уникальности категорий и приведение к категориальному типу
print(data['purpose'].head(10))
print(data['purpose'].head(10).cat.codes)

0          жилье
1     автомобиль
2          жилье
3    образование
4        свадьба
5          жилье
6          жилье
7    образование
8        свадьба
9          жилье
Name: purpose, dtype: category
Categories (5, object): [автомобиль, жилье, недвижимость, образование, свадьба]
0    1
1    0
2    1
3    3
4    4
5    1
6    1
7    3
8    4
9    1
dtype: int8


### Вывод

Выделили 5 наиболее часто встречающихся целей кредита и перевели их в категории: "автомобиль", "жилье", "недвижимость", "образование", "свадьба".

### Шаг 3. Поиск зависимостей.

### Ищем зависимость между наличием детей и возвратом кредита в срок.

In [9]:
print(data['debt'].value_counts()) # Подсчитываем уникальные значения столбца и их количество 
print(data['children'].value_counts()) # Подсчитываем уникальные значения столбца и их количество, обнаружили отрицательное значение
data['children'] = data['children'].apply(abs) # Переводим отрицательные значения в положительные
print('=================================')

data_pivot_children = data[['children', 'debt', 'family_status_id']].pivot_table(index=['children'], columns=['debt'], aggfunc='count') #Подготовили сводную таблицу
data_pivot_children = data_pivot_children.fillna(0) # Заменим отсутствующее значение на 0. 
print(data_pivot_children)
print('=================================')

children_cat_count = data['children'].value_counts() # Создаем переменную, в которой количество должников соответствует числу детей 
def percentage_of_debtors_with_children(row): # Создали функцию, которая рассчитывает процент должников в зависимости от наличия детей 
    childrens = row['family_status_id'].name # Переменная, которая содержит число детей
    row['debtors_children'] = (row['family_status_id', 1] / children_cat_count[childrens])*100 # Из сводной таблицы по очереди берется число должников имеющих, например, 0 детей, делится на число должников в этой группе (также имеющих 0 детей) и умножается на 100%
    return row
data_pivot_children = data_pivot_children.apply(percentage_of_debtors_with_children, axis=1) # Добавляем к сводной таблице столбец с процентом должников, имеющих детей
print(data_pivot_children)

0    19552
1     1725
Name: debt, dtype: int64
 0    14021
 1     4792
 2     2039
 3      328
-1       47
 4       41
 5        9
Name: children, dtype: int64
         family_status_id        
debt                    0       1
children                         
0                 12963.0  1058.0
1                  4397.0   442.0
2                  1845.0   194.0
3                   301.0    27.0
4                    37.0     4.0
5                     9.0     0.0
         family_status_id         debtors_children
debt                    0       1                 
children                                          
0                 12963.0  1058.0         7.545824
1                  4397.0   442.0         9.134119
2                  1845.0   194.0         9.514468
3                   301.0    27.0         8.231707
4                    37.0     4.0         9.756098
5                     9.0     0.0         0.000000


### Вывод

Наиболее безопасными клиентами для банка являются бездетные семьи (7,5%) и семьи с 3 детьми (8,2%). Процент невозврата среди семей с 1, 2 и 4 детьми составляет 9,1, 9,5 и 9,8% соответственно, что превышает средний % невозврата (8%) и говорит о том, что это довольно рискованные группы. Заметили, что между семьями с 0 и 1 детьми осуществляется сильный скачок невозврата с 7 до 9 %, что говорит о том, что наличие детей значимо сказывается на невозврате.

### Исследуем зависимость между семейным положением и возвратом кредита в срок.

In [25]:
print(data['family_status_id'].value_counts()) # Подсчитываем уникальные значения столбца и их количество 
data_pivot_family = data[['debt','family_status_id', 'family_status']].pivot_table(index=['family_status_id'], columns='debt', aggfunc='count')#Создаем сводную таблицу
print(data_pivot_family) 
print('=====================================')
fs_id_count = data['family_status_id'].value_counts()
def percentage_of_debtors_with_family(row):# Создали функцию, которая рассчитывает процент должников в зависимости от семейного статуса
    fs_id = row['family_status'].name # Переменная, которая содержит список семейных статусов
    row['debtors_family'] = (row['family_status', 1] / fs_id_count[fs_id]) * 100 # Из сводной таблицы берется число должников имеющих определенный семейный статус, делится на число должников этого статуса и умножается на 100% 
    return row
data_pivot_family = data_pivot_family.apply(percentage_of_debtors_with_family, axis=1) # Добавляем к сводной таблице столбец с процентом должников, имеющих определенный семейный статус
print(data_pivot_family)

0    12242
1     4117
4     2785
3     1183
2      950
Name: family_status_id, dtype: int64
                 family_status     
debt                         0    1
family_status_id                   
0                        11318  924
1                         3734  383
2                          888   62
3                         1099   84
4                         2513  272
                 family_status        debtors_family
debt                         0      1               
family_status_id                                    
0                      11318.0  924.0       7.547786
1                       3734.0  383.0       9.302890
2                        888.0   62.0       6.526316
3                       1099.0   84.0       7.100592
4                       2513.0  272.0       9.766607


### Вывод

По полученным данным самый высокий процент невозврата у должников, имеющих статус "не женат/не замужем" - 9,8% и "в гражданском браке" - 9,3%. Более низкие проценты невозврата имеют должники из категорий "женат/замужем" (7,5%), "в разводе" (7,1%), "вдовцы" (6,5%), что говорит нам о том, что для банка это более безопасные категории.
Заметно, что между женатыми и неженатыми осуществляется сильный скачок невозврата с 7,5 до 9,8 %, что указывает на то, что семейный статус значимо сказывается на невозврате.

### Выясняем есть ли зависимость между уровнем дохода и возвратом кредита в срок.

In [26]:
print(data['total_income'].describe()) #Посмотрели квартили и медиану, чтобы определить границы категорий ежемесячного дохода

def categorize(row): # Проводим категоризацию. На вход функции попадает ежемесячный доход, а возвращает категорию 
    total_income = row['total_income']
    income_cat = ''
    if total_income > 195751:
         income_cat = 'высокий доход'
    if 195751 >= total_income > 145017:
         income_cat = 'доход выше среднего'
    if 145017 >= total_income > 107620:
         income_cat = 'средний доход'
    if 107620 >= total_income > 50000:
         income_cat = 'доход ниже среднего'
    if total_income <= 50000:
         income_cat = 'низкий доход'
    row['income_cat'] = income_cat
    return row
print('=====================================')
data = data.apply(categorize, axis=1)


income_group_full = data.groupby('income_cat')['debt'].count()
income_group = (data[data['debt'] == 1].groupby('income_cat')['debt'].count() / income_group_full)*100
print(income_group)

count    2.127700e+04
mean     1.652619e+05
std      9.820259e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.450170e+05
75%      1.957510e+05
max      2.265604e+06
Name: total_income, dtype: float64
income_cat
высокий доход          7.144200
доход выше среднего    8.861351
доход ниже среднего    8.121212
низкий доход           6.216216
средний доход          8.504952
Name: debt, dtype: float64


### Вывод

По полученным данным самый высокий процент невозврата у должников, имеющих доход "выше среднего", "средний" и "ниже среднего" - 
8,9, 8,5 и 8,1% соответственно. Наиболее низкие проценты невозврата имеют должники из, казалось бы, противоположных категорий "низких" (6,2%) и "высоких" (7,1%) доходов. По-видимому, эти категории доходов, являются более стабильными, в связи с чем и более безопасными для банка. 
Резкий переход от 6,2% в группе с низким доходом к 8,9% с доходом выше среднего указывает на значимую зависимость между уровнем дохода и возвратом кредита в срок.

### Исследуем влияние разных целей кредита на его возврат в срок.

In [27]:
data_pivot_purpose = data[['debt','purpose', 'gender']].pivot_table(index=['purpose'], columns='debt', aggfunc='count') #Создаем сводную таблицу
print(data_pivot_purpose) 
print('=================================')
purpose_count = data['purpose'].value_counts()
def percentage_of_debtors_with_purpose(row):  # Создали функцию, которая рассчитывает процент должников в зависимости от цели кредита
    purpose_id = row['gender'].name # Переменная, которая содержит список целей
    row['debtors_purpose'] = (row['gender', 1] / purpose_count[purpose_id]) * 100 # Из сводной таблицы берется число должников имеющих определенную цель кредита, делится число должников в конкретной категории и умножается на 100%
    return row
data_pivot_purpose = data_pivot_purpose.apply(percentage_of_debtors_with_purpose, axis=1) # Добавляем к сводной таблице столбец с процентом должников, имеющих определенную цель кредита
print(data_pivot_purpose)

             gender     
debt              0    1
purpose                 
автомобиль     3871  398
жилье          4115  306
недвижимость   5835  471
образование    3611  369
свадьба        2120  181
              gender        debtors_purpose
debt               0      1                
purpose                                    
автомобиль    3871.0  398.0        9.323026
жилье         4115.0  306.0        6.921511
недвижимость  5835.0  471.0        7.469077
образование   3611.0  369.0        9.271357
свадьба       2120.0  181.0        7.866145


### Вывод

По полученным данным самый высокий процент невозврата (9,3%) у должников, имеющих целями кредита "автомобиль" и "образование".
Более низкие проценты невозврата у должников с целями кредита "свадьба" (7,8%), "недвижимость" (7,5%), "жилье" (6,9%), что указывает на то, что для банка это более безопасные категории.
Резкий переход от 6-7% к 9,3% указывает на то, что цель кредита значимо влияет на возврат кредита в срок.

### Шаг 4. Общий вывод

* Наличие детей значимо сказывается на невозврате: наиболее безопасными клиентами для банка являются бездетные семьи (7,5%) и семьи с 3 детьми (8,2%).
* Семейный статус значимо сказывается на возврате кредита в срок. Между женатыми и неженатыми осуществляется сильный скачок невозврата с 7,5 до 9,8 %. Наиболее низкие проценты невозврата имеют должники из категорий "женат/замужем" (7,5%), "в разводе" (7,1%), "вдовцы" (6,5%) и являются более безопасными категориями.
* Определена значимая зависимость между уровнем дохода и возвратом кредита в срок. На это указывает резкий переход от 6,2% невозврата в группе с низким доходом к 8,9% в группе с доходом выше среднего. Наиболее низкие проценты невозврата имеют должники из, казалось бы, противоположных категорий "низких" (6,2%) и "высоких" (7,1%) доходов. По-видимому, эти категории доходов, являются более стабильными, в связи с чем и более безопасными для банка.
* У должников с целями кредита "свадьба", "недвижимость", "жилье" более низкий процент невозврата (7,8, 7,5 и 6,9% соответственно), что указывает на то, что для банка это более безопасные категории. Резкий переход от 6-7% к 9,3% указывает на то, что цель кредита значимо влияет на возврат кредита в срок.